In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Dense, Input, LSTM, Concatenate, Dropout, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

In [2]:
path = r'C:\Users\firoj\OneDrive\Desktop\thesis_test\Thesis_test_01\Final_data'
os.chdir(path)

In [3]:
df = pd.read_csv('main_data.csv')
df.head()

,ais_timestamp,vessel_mmsi,vessel_length,lon,lat,distance,speed,destination_lat,destination_lon,remaining_distance,lock_status,water_level_m,sig_m,wave_tp,wind_m/s,sine,cosine,target
0,01.05.2023 07:08,210090000,88,8.545379,53.56142,0,110,53.49468,8.504,7909.105180,1,5.02,0.3666,356.8,3.5,0.484810,-0.874620,27.00
1,01.05.2023 07:11,210090000,88,8.554007,53.55247,1150,112,53.49468,8.504,7226.372365,1,5.06,0.3575,347.8,3.1,0.500000,-0.866025,24.00
2,01.05.2023 07:14,210090000,88,8.562378,53.54377,1110,111,53.49468,8.504,6685.145437,1,5.10,0.3716,359.2,1.3,0.484810,-0.874620,20.98
3,01.05.2023 07:17,210090000,88,8.567384,53.53481,1050,110,53.49468,8.504,6121.669271,1,5.14,0.3833,362.5,2.6,-0.052336,-0.998630,18.00
4,01.05.2023 07:20,210090000,88,8.562150,53.52593,1050,110,53.49468,8.504,5182.656801,1,5.18,0.3616,346.5,2.6,-0.681998,-0.731354,15.00


In [4]:
df.shape

(49506, 18)

In [5]:
df.columns

Index(['ais_timestamp', 'vessel_mmsi', 'vessel_length', 'lon', 'lat',
       'distance', 'speed', 'destination_lat', 'destination_lon',
       'remaining_distance', 'lock_status', 'water_level_m', 'sig_m',
       'wave_tp', 'wind_m/s', 'sine', 'cosine', 'target'],
      dtype='object')

In [6]:
final_df = df.drop(columns=['ais_timestamp',
                            'vessel_mmsi'])

In [7]:
final_df.head()

,vessel_length,lon,lat,distance,speed,destination_lat,destination_lon,remaining_distance,lock_status,water_level_m,sig_m,wave_tp,wind_m/s,sine,cosine,target
0,88,8.545379,53.56142,0,110,53.49468,8.504,7909.105180,1,5.02,0.3666,356.8,3.5,0.484810,-0.874620,27.00
1,88,8.554007,53.55247,1150,112,53.49468,8.504,7226.372365,1,5.06,0.3575,347.8,3.1,0.500000,-0.866025,24.00
2,88,8.562378,53.54377,1110,111,53.49468,8.504,6685.145437,1,5.10,0.3716,359.2,1.3,0.484810,-0.874620,20.98
3,88,8.567384,53.53481,1050,110,53.49468,8.504,6121.669271,1,5.14,0.3833,362.5,2.6,-0.052336,-0.998630,18.00
4,88,8.562150,53.52593,1050,110,53.49468,8.504,5182.656801,1,5.18,0.3616,346.5,2.6,-0.681998,-0.731354,15.00


In [8]:
final_df.columns

Index(['vessel_length', 'lon', 'lat', 'distance', 'speed', 'destination_lat',
       'destination_lon', 'remaining_distance', 'lock_status', 'water_level_m',
       'sig_m', 'wave_tp', 'wind_m/s', 'sine', 'cosine', 'target'],
      dtype='object')

In [9]:
# Exclude the target column and any other columns we don't want to standardize
columns_to_exclude = ['lock_status', 'sine', 'cosine', 'target']  
features_to_standardize = final_df.columns.difference(columns_to_exclude)

# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize the features
final_df[features_to_standardize] = scaler.fit_transform(final_df[features_to_standardize])

In [10]:
final_df.head()

,vessel_length,lon,lat,distance,speed,destination_lat,destination_lon,remaining_distance,lock_status,water_level_m,sig_m,wave_tp,wind_m/s,sine,cosine,target
0,0.804927,-0.582344,1.024065,-0.889429,1.528653,0.869137,-0.744634,0.493282,1,0.498543,-0.765016,0.025329,-0.619514,0.484810,-0.874620,27.00
1,0.804927,-0.542961,0.999784,1.905452,1.579116,0.869137,-0.744634,0.356651,1,0.519692,-0.789329,-0.101788,-0.779312,0.500000,-0.866025,24.00
2,0.804927,-0.504750,0.976180,1.808238,1.553884,0.869137,-0.744634,0.248338,1,0.540841,-0.751658,0.059227,-1.498407,0.484810,-0.874620,20.98
3,0.804927,-0.481900,0.951871,1.662419,1.528653,0.869137,-0.744634,0.135573,1,0.561990,-0.720399,0.105837,-0.979061,-0.052336,-0.998630,18.00
4,0.804927,-0.505791,0.927779,1.662419,1.528653,0.869137,-0.744634,-0.052346,1,0.583138,-0.778375,-0.120149,-0.979061,-0.681998,-0.731354,15.00


In [11]:
final_df.columns

Index(['vessel_length', 'lon', 'lat', 'distance', 'speed', 'destination_lat',
       'destination_lon', 'remaining_distance', 'lock_status', 'water_level_m',
       'sig_m', 'wave_tp', 'wind_m/s', 'sine', 'cosine', 'target'],
      dtype='object')

In [12]:
# features and target variable
X = final_df.drop('target', axis=1)
y = final_df['target']

# Split the dataset into training and temp (validation + test) sets
# Let's say we want 70% of data in the training set and 30% in the temp set
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the temp set equally into validation and test sets
# This will result in 15% of the original data in the validation set and 15% in the test set
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
# Define the base directory path
base_dir = r'C:\Users\firoj\OneDrive\Desktop\thesis_test\Thesis_test_01\Feature_matrix\All\\'  # Note the added backslash at the end
os.chdir(base_dir)

In [14]:
# Save the feature sets
np.save(base_dir + 'X_train.npy', X_train)
np.save(base_dir + 'X_val.npy', X_val)
np.save(base_dir + 'X_test.npy', X_test)

# Save the target sets
np.save(base_dir + 'y_train.npy', y_train)
np.save(base_dir + 'y_val.npy', y_val)
np.save(base_dir + 'y_test.npy', y_test)